# Airline fare price prediction

Using the dataset provided by [lalit_joshi](https://www.kaggle.com/datasets/lalitjoshi89/airlinepriceprediction)

In [11]:
# Importing libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [12]:
# Loading "raw" data from csv.
data = pd.read_csv(r'dataset/airlines_data.csv')
data.info

<bound method DataFrame.info of      ;Airline_Name;Date_of_Journey;Source;Destination;Dept_Time;Total_Stops;Duration_of_Flight;Arr_Time;Fare
0     0;AirAsia;26/02/2022;Kolkata;Mumbai;13:30;1 st...                                                     
1     1;AirAsia;26/02/2022;Kolkata;Mumbai;9:05;2 sto...                                                     
2     2;AirAsia;26/02/2022;Kolkata;Mumbai;16:15;1 st...                                                     
3     3;AirAsia;26/02/2022;Kolkata;Mumbai;23:40;1 st...                                                     
4     4;AirAsia;26/02/2022;Kolkata;Mumbai;20:00;1 st...                                                     
...                                                 ...                                                     
2016  2019;Vistara;10/7/22;Mumbai;Chennai;6:20;2 Sto...                                                     
2017  2020;Vistara;10/7/22;Mumbai;Chennai;11:25;2 St...                                         

### The Dataset

The data comes in a excel file format which I re-exported as a csv directly from Excel (note this could have been done using python too.)

In the data we have 10 variables and 2021 features. But we get all in 1 column due to the file being delimeted by semicolon. Lets fix that!

In [13]:
# Formating the columns to display the dataframe correctly.
data = pd.read_csv(r'dataset/airlines_data.csv', delimiter=';')
data

,Unnamed: 0,Airline_Name,Date_of_Journey,Source,Destination,Dept_Time,Total_Stops,Duration_of_Flight,Arr_Time,Fare
0,0,AirAsia,26/02/2022,Kolkata,Mumbai,13:30,1 stop,07 h 05 m,20:35,3 379
1,1,AirAsia,26/02/2022,Kolkata,Mumbai,9:05,2 stop,13 h 10 m,22:15,3 379
2,2,AirAsia,26/02/2022,Kolkata,Mumbai,16:15,1 stop,08 h 20 m,0:35,3 379
3,3,AirAsia,26/02/2022,Kolkata,Mumbai,23:40,1 stop,06 h 55 m,6:35,3 379
4,4,AirAsia,26/02/2022,Kolkata,Mumbai,20:00,1 stop,10 h 35 m,6:35,3 379
...,...,...,...,...,...,...,...,...,...,...
2016,2019,Vistara,10/7/22,Mumbai,Chennai,6:20,2 Stop,13h 55m,20:15,15 192
2017,2020,Vistara,10/7/22,Mumbai,Chennai,11:25,2 Stop,11h 20m,22:45,16 442
2018,2021,Vistara,10/7/22,Mumbai,Chennai,6:45,2 Stop,13h 30m,20:15,16 442
2019,2022,Vistara,10/7/22,Mumbai,Chennai,9:05,2 Stop,11h 10m,20:15,17 282


In [15]:
# Checking data types.
data.dtypes

Unnamed: 0             int64
Airline_Name          object
Date_of_Journey       object
Source                object
Destination           object
Dept_Time             object
Total_Stops           object
Duration_of_Flight    object
Arr_Time              object
Fare                  object
dtype: object

In [16]:
# Checking for missing values.
data.isnull().sum()

Unnamed: 0            0
Airline_Name          0
Date_of_Journey       0
Source                0
Destination           0
Dept_Time             0
Total_Stops           0
Duration_of_Flight    0
Arr_Time              0
Fare                  0
dtype: int64

### **Good!** and **Bad!**

Now there are some obvious observations that need to be address such as:

- The first `Unnamed` column is redundant  and needs to be drop as it seems to be a count or represntation of the index.
- The is inconsistenacy on the date format on the `Date_of_Journey` column
- The `Total_Stops` Column should be converted to numerical, the word 'Stop' is not necessary.

A lot of our data is not the proper type such as:
- `Date_of_Journey` should be `date64`
- `Dept_Time`, `Arr_time`, `Duration_of_Flight` should be `timedelta`
- `Total_Stops` should be `int` (and modified to be numerical as mentiona above)
- `Fare` should be `int`

It would be a good idea to change the name of the columns to shorter names.

In [17]:
# Droping "Unnamed" column.
data.drop(data.columns[0], axis=1, inplace=True)
data.head()

,Airline_Name,Date_of_Journey,Source,Destination,Dept_Time,Total_Stops,Duration_of_Flight,Arr_Time,Fare
0,AirAsia,26/02/2022,Kolkata,Mumbai,13:30,1 stop,07 h 05 m,20:35,3 379
1,AirAsia,26/02/2022,Kolkata,Mumbai,9:05,2 stop,13 h 10 m,22:15,3 379
2,AirAsia,26/02/2022,Kolkata,Mumbai,16:15,1 stop,08 h 20 m,0:35,3 379
3,AirAsia,26/02/2022,Kolkata,Mumbai,23:40,1 stop,06 h 55 m,6:35,3 379
4,AirAsia,26/02/2022,Kolkata,Mumbai,20:00,1 stop,10 h 35 m,6:35,3 379


In [18]:
# Renaming and reordering columns.
new_cols = ["airline_name", "flight_date", "flight_dep", "flight_arr", "dep_time", "total_stops", "flight_time", "arr_time", "flight_fare"]
data.columns = new_cols

# Reordering columns.
new_cols_ord = ["airline_name", "flight_date", "flight_dep", "dep_time", "flight_arr", "arr_time", "total_stops","flight_time", "flight_fare"]
data = data.reindex(columns=new_cols_ord)

data.head()

,airline_name,flight_date,flight_dep,dep_time,flight_arr,arr_time,total_stops,flight_time,flight_fare
0,AirAsia,26/02/2022,Kolkata,13:30,Mumbai,20:35,1 stop,07 h 05 m,3 379
1,AirAsia,26/02/2022,Kolkata,9:05,Mumbai,22:15,2 stop,13 h 10 m,3 379
2,AirAsia,26/02/2022,Kolkata,16:15,Mumbai,0:35,1 stop,08 h 20 m,3 379
3,AirAsia,26/02/2022,Kolkata,23:40,Mumbai,6:35,1 stop,06 h 55 m,3 379
4,AirAsia,26/02/2022,Kolkata,20:00,Mumbai,6:35,1 stop,10 h 35 m,3 379


In [19]:
# Changing data type of flight_date to desire format: Y-M-D.
data['flight_date'] = pd.to_datetime(data['flight_date'])
data['flight_date']

0      2022-02-26
1      2022-02-26
2      2022-02-26
3      2022-02-26
4      2022-02-26
          ...    
2016   2022-10-07
2017   2022-10-07
2018   2022-10-07
2019   2022-10-07
2020   2022-10-07
Name: flight_date, Length: 2021, dtype: datetime64[ns]

In [20]:
# Changing flight departure and arrive data type.
for i in data['dep_time']:
     i = datetime.strptime(i, '%H:%M')
data['dep_time']

0       13:30
1        9:05
2       16:15
3       23:40
4       20:00
        ...  
2016     6:20
2017    11:25
2018     6:45
2019     9:05
2020     7:00
Name: dep_time, Length: 2021, dtype: object

In [98]:
# Corting string to numerical values in total_flights column
data.loc[data['total_stops'].str.contains('1', na=False)] = 1
data.loc[data['total_stops'].str.contains('2', na=False)] = 2
data.loc[data['total_stops'].str.contains('3', na=False)] = 3
data.loc[data['total_stops'].str.contains('Non', na=False)] = 0

In [94]:
"""for i in data['total_stops']:
    if data.loc[data['total_stops'].str.contains('1', na=False)]:
        i = 1
    elif data.loc[data['total_stops'].str.contains('2', na=False)]:
        i = 2
    elif data.loc[data['total_stops'].str.contains('3', na=False)]:
        i = 3
    else:
        i = 0"""

#data['total_stops']

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Changing flight time data type to time.

for i in data['flight_time']:
    pd.to_timedelta(data['flight_time'], unit='h', errors='raise')

In [ ]:
# Converting fare column to int data type.
data['fare'] = pd.to_numeric(data['fare'])
data['fare'].dtype